# Feature Pipeline

The feature pipeline will be scheduled to run once per day, and it performs the following tasks:
1. Reads today's PM2.5 measurements.
2. Reads today's weather data measurements.
3. Reads the weather forecast data for the next 7 days.
4. Insert all of this data into the air quality and weather feature groups, respectively.

In [48]:
import datetime
import pandas as pd
import hopsworks
import datetime
import util
import json
import warnings
warnings.filterwarnings("ignore")

In [49]:
import dotenv

dotenv.load_dotenv()

True

In [50]:
# Connect to the feature store
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

# Extract the location information
country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

# Get the current date
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)
two_days_ago = today - datetime.timedelta(days=2)
three_days_ago = today - datetime.timedelta(days=3)
location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1157270
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "sweden", "city": "stockholm", "street": "stockholm-st-eriksgatan-83", "aqicn_url": "https://api.waqi.info/feed/@10523", "latitude": 59.33, "longitude": 18.07}'

In [51]:
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

In [52]:
air_quality_data=air_quality_fg.read()

# Get the PM2.5 values for the last 3 days
pm25_1_days_before = air_quality_data[air_quality_data['date'] == yesterday.strftime('%Y-%m-%d')]['pm25'].values[0]
pm25_2_days_before = air_quality_data[air_quality_data['date'] == two_days_ago.strftime('%Y-%m-%d')]['pm25'].values[0]
pm25_3_days_before = air_quality_data[air_quality_data['date'] == three_days_ago.strftime('%Y-%m-%d')]['pm25'].values[0]

pm25_1_days_before, pm25_2_days_before, pm25_3_days_before

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.26s) 


(14.0, 13.0, 22.0)

In [53]:
# Get the current weather
aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df


,pm25,country,city,street,date,url
0,16.0,sweden,stockholm,stockholm-st-eriksgatan-83,2024-11-18,https://api.waqi.info/feed/@10523


In [54]:
# Add the PM2.5 values from the previous 3 days
aq_today_df["pm25_1_days_before"] = pm25_1_days_before
aq_today_df["pm25_2_days_before"] = pm25_2_days_before
aq_today_df["pm25_3_days_before"] = pm25_3_days_before
aq_today_df["pm25_avg_3_days_before"] = (pm25_1_days_before + pm25_2_days_before + pm25_3_days_before) / 3

# Cast the columns to float32
aq_today_df["pm25_avg_3_days_before"] = aq_today_df["pm25_avg_3_days_before"].astype("float32")
aq_today_df["pm25_1_days_before"] = aq_today_df["pm25_1_days_before"].astype("float32")
aq_today_df["pm25_2_days_before"] = aq_today_df["pm25_2_days_before"].astype("float32")
aq_today_df["pm25_3_days_before"] = aq_today_df["pm25_3_days_before"].astype("float32")

aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   pm25                    1 non-null      float32       
 1   country                 1 non-null      object        
 2   city                    1 non-null      object        
 3   street                  1 non-null      object        
 4   date                    1 non-null      datetime64[ns]
 5   url                     1 non-null      object        
 6   pm25_1_days_before      1 non-null      float32       
 7   pm25_2_days_before      1 non-null      float32       
 8   pm25_3_days_before      1 non-null      float32       
 9   pm25_avg_3_days_before  1 non-null      float32       
dtypes: datetime64[ns](1), float32(5), object(4)
memory usage: 192.0+ bytes


In [55]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# Gets forecast for next 10 days
# Replace  hourly forecasts with a single daily forecast
# Get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])     #To date format

daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-18,2.80,0.0,11.542478,273.576263,stockholm
1,2024-11-19,1.95,0.0,6.034700,107.354111,stockholm
2,2024-11-20,1.60,0.0,18.947083,14.300317,stockholm
3,2024-11-21,-2.65,0.7,28.666941,305.286835,stockholm
4,2024-11-22,-4.90,0.0,16.622490,287.650208,stockholm
5,2024-11-23,-4.10,0.0,10.805999,271.909119,stockholm
6,2024-11-24,3.75,0.0,18.899143,139.635544,stockholm
7,2024-11-25,6.85,0.0,23.400002,194.250061,stockholm
8,2024-11-26,3.50,0.1,15.463244,282.094727,stockholm
9,2024-11-27,0.20,0.0,14.003029,287.969238,stockholm


In [56]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 452.0+ bytes


In [57]:
# Insert Air Quality to the feature group
air_quality_fg.insert(aq_today_df)

2024-11-18 13:03:04,804 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1157270/fs/1147973/fg/1353047


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1157270/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x30f29c530>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 676883
         }
       },
       "result": {
         "observed_value": 16.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-18T12:03:04.000804Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsucc

In [58]:
# Insert Weather to the feature group
weather_fg.insert(daily_df)     #Insert weather data to feature store

2024-11-18 13:03:15,770 INFO: 	2 expectation(s) included in expectation_suite.
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1157270/fs/1147973/fg/1354064


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1157270/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x30f32d760>,
 {
   "success": false,
   "results": [
     {
       "success": false,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": 0.0,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 677899
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-18T12:03:15.000770Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_betwee